**Given the path to the folder containing all the datasets(dataset_path), this script will align the slam output with the ground truth and perform the error metrics estimation and generate a file containing the error metrics.** 

## Import Modules

In [1]:
import os
import json
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from typing import List
import math
from matplotlib.patches import Patch
from distinctipy import get_colors # to get unique colors
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from itertools import product
from numpy.linalg import LinAlgError, eigvalsh
from scipy.spatial.transform import Rotation
from scipy.spatial import procrustes
from scipy.spatial.distance import directed_hausdorff
from scipy.spatial.distance import cdist

# save matplotlib plots without displaying
%matplotlib agg

# Global Data and Functions

In [2]:
# BOP YCBV dataset
dataset_info = {'1': 'master_chef_can',
                '2': 'cracker_box',
                '3': 'sugar_box',
                '4': 'tomato_soup_can',
                '5': 'mustard_bottle',
                '6': 'tuna_fish_can',
                '7': 'pudding_box',
                '8': 'gelatin_box',
                '9': 'potted_meat_can',
                '10': 'banana',
                '11': 'pitcher_base',
                '12': 'bleach_cleanser',
                '13': 'bowl',
                '14': 'mug',
                '15': 'power_drill',
                '16': 'wood_block',
                '17': 'scissors',
                '18': 'large_marker',
                '19': 'large_clamp',
                '20': 'extra_large_clamp',
                '21': 'foam_brick'}

## Function to plot ellipsoids

In [3]:
# Reference - https://github.com/qcr/quadricslam/blob/master/src/quadricslam/visualisation.py
def plot_ellipsoid(pose: np.ndarray, radii: np.ndarray, ax: matplotlib.figure.Axes, color):
    # Generate ellipsoid of appropriate size at origin
    SZ = 50
    radii = np.abs(radii)
    u, v = np.linspace(0, 2 * np.pi, SZ), np.linspace(0, np.pi, SZ)
    x, y, z = (radii[0] * np.outer(np.cos(u), np.sin(v)),
               radii[1] * np.outer(np.sin(u), np.sin(v)),
               radii[2] * np.outer(np.ones_like(u), np.cos(v)))

    # Rotate the ellipsoid, then translate to centroid
    ps = pose @ np.vstack([
        x.reshape(-1),
        y.reshape(-1),
        z.reshape(-1),
        np.ones(z.reshape(-1).shape)
    ])

    # Plot the ellipsoid
    ax.plot_wireframe(
        ps[0, :].reshape(SZ, SZ),
        ps[1, :].reshape(SZ, SZ),
        ps[2, :].reshape(SZ, SZ),
        rstride=4,
        cstride=4,
        color=color,
        linewidth=0.5,
    )## Function to plot ellipsoids

## Function to plot cuboids

In [4]:
def plot_cuboid(pose: np.ndarray, size: np.ndarray, ax: matplotlib.figure.Axes, color):
    # Get all 8 corner points of the cuboid
    #vertices = np.array(list(product(*zip(pose[:3, -1] - 0.5 * size, pose[:3, -1] + 0.5 * size))))
    vertices = np.array(list(product(*zip(- 0.5 * size, + 0.5 * size))))
    
    # Transform the cuboid's vertices using the pose matrix
    t_vertices = np.dot(pose[:3, :3], vertices.T).T + pose[:3, -1]
    
    # Define the edges of the cuboid using the vertices
    edges = [
        [t_vertices[0], t_vertices[1]], [t_vertices[1], t_vertices[5]], [t_vertices[5], t_vertices[4]],
        [t_vertices[4], t_vertices[0]], [t_vertices[7], t_vertices[6]], [t_vertices[6], t_vertices[2]],
        [t_vertices[2], t_vertices[3]], [t_vertices[3], t_vertices[7]], [t_vertices[0], t_vertices[2]],
        [t_vertices[1], t_vertices[3]], [t_vertices[4], t_vertices[6]], [t_vertices[5], t_vertices[7]]
    ]
    
    ax.add_collection3d(Line3DCollection(edges, colors=color, linewidths=2))

## Function to plot trajectory

In [5]:
def plot_traj(data: List, ax: matplotlib.figure.Axes, length = 50, c:str = 'cyan', label = None):
    
    # to plot trajectory
    traj = []
    
    for idx in range(len(data)):
        # position
#         ax.scatter(data[idx][0,-1], data[idx][1,-1], data[idx][2,-1], c='r', marker='*')
        # orientation
        if idx % 30 == 0:
            # X
            ax.quiver(data[idx][0,-1], data[idx][1,-1], data[idx][2,-1],
                      data[idx][0, 0], data[idx][1, 0], data[idx][2, 0],
                      color='r', length=length, linewidth=0.2, alpha=1)
            # Y
            ax.quiver(data[idx][0,-1], data[idx][1,-1], data[idx][2,-1],
                      data[idx][0, 1], data[idx][1, 1], data[idx][2, 1],
                      color='g', length=length, linewidth=0.2, alpha=1)
            # Z
            ax.quiver(data[idx][0,-1], data[idx][1,-1], data[idx][2,-1],
                      data[idx][0, 2], data[idx][1, 2], data[idx][2, 2],
                      color='b', length=length, linewidth=0.2, alpha=1)
        
        traj.append([data[idx][0,-1], data[idx][1,-1], data[idx][2,-1]])
        
    #trajectory
    if c != 'cyan':
        traj = np.array(traj)
        if label==None:
            ax.plot(traj[:, 0], traj[:, 1], traj[:, 2], color=c)
        else:
            ax.plot(traj[:, 0], traj[:, 1], traj[:, 2], color=c, label = label)
        
    
    return None

# Functions for Error Calculation of the data 

### Euclidean Distance

In [6]:
# Given 2 centroids, it will output the euclidean distance between the 2 points
def euclidean_distance(centroid1, centroid2):
    return np.linalg.norm(centroid1 - centroid2)

### Rotation Error

In [7]:
## rotation matrix to quaternion conversion
def matrix_to_quaternion(matrix):
    # Extract the 3x3 rotation matrix from the 4x4 transformation matrix
    rotation_matrix = matrix[:3, :3]

    # Ensure the rotation matrix is orthogonal (if necessary)
    if not np.allclose(np.linalg.det(rotation_matrix), 1.0):
        rotation_matrix = rotation_matrix / np.cbrt(np.linalg.det(rotation_matrix))

    # Create a Rotation object from the rotation matrix
    rotation = Rotation.from_matrix(rotation_matrix)

    # Get the quaternion representation
    quaternion = rotation.as_quat()

    return quaternion


## quaternion representation of the rotation matrix as input
def rotation_error_quaternion(quat1, quat2):
    r1 = Rotation(quat1)
    r2 = Rotation(quat2)

    # Convert to unit quaternions if necessary
    if not np.isclose(np.linalg.norm(r1.as_quat()), 1.0):
        r1 = r1.normalized()
    if not np.isclose(np.linalg.norm(r2.as_quat()), 1.0):
        r2 = r2.normalized()
    # Compute the rotation error (angle between the quaternions)
    error_angle = 2.0 * np.arccos(np.abs(np.dot(r1.as_quat(), r2.as_quat())))
    return error_angle

### Procrustes Analysis

In [8]:
def procrustes_analysis(points1, points2):
    set1, set2, disparity = procrustes(points1, points2)
    return set1, set2, disparity

### Fréchet Distance

In [9]:
def frechet_distance(curve1, curve2):
    
    # Compute the directed Hausdorff distance (equivalent to Fréchet Distance)
    distance_1_to_2, _, _ = directed_hausdorff(curve1, curve2)
    distance_2_to_1, _, _ = directed_hausdorff(curve2, curve1)

    # Return the maximum of the two directed Hausdorff distances (Fréchet Distance)
    frechet_distance = max(distance_1_to_2, distance_2_to_1)
    return frechet_distance

### Chamfer Distance

In [10]:
def chamfer_distance(point_cloud1, point_cloud2):
    return np.mean(np.min(cdist(point_cloud1, point_cloud2),axis=0)) + np.mean(np.min(cdist(point_cloud2, point_cloud1), axis=0))

### Percentage of Intersection Volume

In [11]:
# function to generate points within GT cuboid
def generate_points_within_cuboid(centroid, dimensions, rotation_matrix, num_points=1000):
    # Extract centroid coordinates
    cx, cy, cz = centroid
    
    # Extract dimensions of the cuboid
    Lx, Ly, Lz = dimensions
    
    # Generate random points within the unit cube
    points_in_unit_cube = np.random.rand(num_points, 3)
    
    # Scale and translate points to fit within the cuboid
    points_in_cuboid = points_in_unit_cube * np.array([Lx, Ly, Lz]) - np.array([Lx / 2, Ly / 2, Lz / 2])
    
    # Apply the rotation to the points
    rotated_points = np.dot(points_in_cuboid, rotation_matrix.T)
    
    # Translate the points to the cuboid centroid
    translated_points = rotated_points + np.array([cx, cy, cz])
    
    return translated_points

# function to check if the point lies within the ellipsoid
def is_point_inside_ellipsoid(point, centroid, radii, rotation_matrix):
    # Convert the point to the ellipsoid's local coordinate system
    local_point = point - centroid

    # Apply the inverse rotation to bring the point to the ellipsoid's local orientation
    local_point_rotated = np.dot(rotation_matrix.T, local_point)

    # Normalize the point by dividing its coordinates by the semi-axes lengths of the ellipsoid
    normalized_point = local_point_rotated / radii

    # Check if the normalized point lies within the unit sphere
    return np.linalg.norm(normalized_point) <= 1.0

### Rotation Error Correction Function

In [12]:
def threshold_to_closest_multiple_of_90(number):
    # Calculate the remainder when dividing the number by 90
    remainder = number % 90
    
    # Determine whether to round up or down based on the remainder
    if remainder < 45:
        return number - remainder
    else:
        return number + (90 - remainder)

# Function for Postprocessing the data

In [13]:
# dataset -> path to the dataset folder
# plottings -> the bool that controls the plots to be saved or not
# batch -> if batch, then read the batch output file
def postprocessing(dataset:str, plottings:bool, batch:bool):
    
    # directory to save images
    if batch:
        images_save_directory = os.path.join(dataset, 'quadric_slam_result/images_batch')
    else:
        images_save_directory = os.path.join(dataset, 'quadric_slam_result/images_incre')
    if not os.path.exists(images_save_directory):
        os.makedirs(images_save_directory)
    
    
    ################# GROUND TRUTH CAMERA POSE #################
    ## reading the ground truth scene_camera.json file
    f = open(dataset + '/scene_camera.json')
    camera_pose = json.load(f)
    
    ## used for plotting
    x_min = 0
    x_max = 0
    y_min = 0
    y_max = 0
    z_min = 0
    z_max = 0
    
    ## The camera poses are in the format w2c(World with respect to camera)
    ## We need c2w(Camera with respect to world)
    
    gt_traj = [] ## to store the c2w as 4x4 numpy arrays within a list 

    for c in camera_pose:
        
        # converting from json raw format to a numpy array
        world2camera = np.column_stack((np.array(camera_pose[c]['cam_R_w2c']).reshape(3,3),
                                        np.array(camera_pose[c]['cam_t_w2c']).reshape(3,1)))
        world2camera = np.row_stack((world2camera, np.array([0. , 0., 0., 1.])))

        # In the dataset, the transformation is w2c(world with respect to the camera)
        # we need to change it to camera with respect to the world.
        corrected_pose = np.linalg.inv(world2camera)
        gt_traj.append(corrected_pose)
        
        x_min = min(x_min, corrected_pose[0,-1])
        x_max = max(x_max, corrected_pose[0,-1])
        y_min = min(y_min, corrected_pose[1,-1])
        y_max = max(y_max, corrected_pose[1,-1])
        z_min = min(z_min, corrected_pose[2,-1])
        z_max = max(z_max, corrected_pose[2,-1])
    ################# GROUND TRUTH CAMERA POSE #################
    
    
    ################# GROUND TRUTH OBJECT POSE #################
    ## reading the ground truth scene_gt.json file
    f = open(dataset + '/scene_gt.json')
    object_pose = json.load(f)
    
    ## Loading ground truth obejct models and assigning colors
    f = open(os.path.dirname(dataset) + '/models_info.json')
    model_data = json.load(f)

    classes = [] ## which all classes are present in the current datset

    for o in range(len(object_pose['1'])):
        classes.append(object_pose['1'][o]['obj_id'])
    
    # get unique colors for each class
    colors = get_colors(len(object_pose['1']))    
    
    # to store all object poses
    gt_pose_data_all = [[] for _ in range(len(object_pose['1']))]

    idx = 0
    for c in camera_pose:

        for o in range(len(object_pose['1'])):

            object2camera = np.column_stack((np.array(object_pose[c][o]['cam_R_m2c']).reshape(3,3),
                                             np.array(object_pose[c][o]['cam_t_m2c']).reshape(3,1)))
            object2camera = np.row_stack((object2camera, np.array([0. , 0., 0.,1.])))

            # given W with respect to C and M with respect to C
            # We need M with respect to W. In other words,
            # we need W_T_M = inv(C_T_W) @ C_T_M = W_T_C @ C_T_M 
            gt_pose_data_all[o].append(np.dot(gt_traj[idx], object2camera))

        idx+=1
    
    # averaging and getting a single position for the object pose
    gt_pose = [] # to store the pose of each object

    for obj in gt_pose_data_all:
        arrays_stack = np.stack(obj)
        averaged_pose = np.mean(arrays_stack, axis=0)
        gt_pose.append(averaged_pose)
        
        x_min = min(x_min, averaged_pose[0,-1])
        x_max = max(x_max, averaged_pose[0,-1])
        y_min = min(y_min, averaged_pose[1,-1])
        y_max = max(y_max, averaged_pose[1,-1])
        z_min = min(z_min, averaged_pose[2,-1])
        z_max = max(z_max, averaged_pose[2,-1])
    
    ################# GROUND TRUTH OBJECT POSE #################
    
    
    ################# GROUND TRUTH SCENE PLOTTING #################
    if plottings:
        fig = plt.figure(figsize=(10,6))
        ax = fig.add_subplot(111, projection='3d')
        ax.set_xlabel('X Axis')
        ax.set_ylabel('Y Axis')
        ax.set_zlabel('Z Axis')
        ax.set_xlim((x_min-100, x_max+100))
        ax.set_ylim((y_min-100, y_max+100))
        ax.set_zlim((z_min-100, z_max+100))
        plot_traj(gt_traj, ax, c = 'blue', length=200)

        # plotting as poses
        for o in range(len(object_pose['1'])):
            plot_traj([gt_pose[o]], ax, length=200)


        # plotting as ellipsoids and cuboids
        for o in range(len(object_pose['1'])):
            class_id = str(classes[o])
            radii = np.array([model_data[class_id]['size_x'],
                              model_data[class_id]['size_y'],
                              model_data[class_id]['size_z']])
            # plotting as ellipsoids
            # plot_ellipsoid(gt_pose[o], radii/2, ax, colors[o])
            # plotting as cuboids
            plot_cuboid(gt_pose[o], radii, ax, colors[o])

        ax.legend(handles=[
                Patch(facecolor=c, edgecolor=c, label=dataset_info[str(l)]) for l, c in zip(classes, colors)
            ])
        ax.grid(False)
        plt.savefig(images_save_directory + '/ground_truth_scene.png')
        plt.close()
        
    ################# GROUND TRUTH SCENE PLOTTING #################
    
    ################# ESTIMATED CAMERA POSE #################
    
    ## reading output file
    if batch:
        output_path = dataset + "/quadric_slam_result/output_batch.json"
    else:
        output_path = dataset + "/quadric_slam_result/output_incre.json"

    with open(output_path, 'r') as file:
        output_data = json.load(file)
    
    ## unaligned camera poses
    unaligned_traj = []

    for k,v in output_data['poses'].items():
        unaligned_traj.append(np.array(output_data['poses'][k]))
    
    ## unaligned trajectories
    unaligned_quad = output_data['quadrics']

    for k,v in unaligned_quad.items():
        unaligned_quad[k]['pose'] = np.array(v['pose'])
        unaligned_quad[k]['centroid'] = np.array(v['centroid'])
        unaligned_quad[k]['radii'] = np.abs(np.array(v['radii']))
        
    # to store the object class label for each quadric key
    labels = output_data['labels']
    
    ## Aligning the estimated camera pose with origin
    est_traj_origin = np.linalg.inv(unaligned_traj[0])

    origin_aligned_traj = []

    for traj in unaligned_traj:
        origin_aligned_traj.append(np.dot(est_traj_origin, traj))
        
    ## Aligning the estimated camera pose with ground truth
    aligned_traj = []

    for traj in origin_aligned_traj:
        aligned_traj.append(np.dot(gt_traj[0], traj))
        
    ## Aligning the estimated object pose with origin
    origin_aligned_quad = unaligned_quad

    for k,v in origin_aligned_quad.items():
        origin_aligned_quad[k]['pose'] = np.dot(est_traj_origin, v['pose'])
        
    ## Aligning the estimated object pose with ground truth
    aligned_quad = origin_aligned_quad

    for k,v in aligned_quad.items():
        aligned_quad[k]['pose'] = np.dot(gt_traj[0], v['pose'])
        
    ################# ESTIMATED TRAJECTORY PLOTTING #################
    if plottings:
        fig = plt.figure(figsize=(10,6))
        ax = fig.add_subplot(111, projection='3d')
        ax.set_xlabel('X Axis')
        ax.set_ylabel('Y Axis')
        ax.set_zlabel('Z Axis')
        ax.set_xlim((x_min-100, x_max+100))
        ax.set_ylim((y_min-100, y_max+100))
        ax.set_zlim((z_min-100, z_max+100))

        plot_traj(gt_traj, ax, c = 'blue', label="ground truth", length=200)
        plot_traj(aligned_traj, ax, c = 'green', label="estimated", length=200)

        ax.legend()
        ax.grid(False)
        plt.savefig(images_save_directory + '/aligned_trajectories.png')
        plt.close()
    ################# ESTIMATED TRAJECTORY PLOTTING #################
    
    
    ################# ESTIMATED SCENE PLOTTING #################
    if plottings:
        fig = plt.figure(figsize=(10,6))
        ax = fig.add_subplot(111, projection='3d')
        ax.set_xlabel('X Axis')
        ax.set_ylabel('Y Axis')
        ax.set_zlabel('Z Axis')
        ax.set_xlim((x_min-100, x_max+100))
        ax.set_ylim((y_min-100, y_max+100))
        ax.set_zlim((z_min-100, z_max+100))

        ## plotting trajectories
        plot_traj(gt_traj, ax, c = 'blue', label="ground truth", length=200)
        plot_traj(aligned_traj, ax, c = 'green', label="estimated", length=200)

        # plotting GT object pose as cuboid
        for o in range(len(object_pose['1'])):
            plot_traj([gt_pose[o]], ax, length=200)

            class_id = str(classes[o])
            radii = np.array([model_data[class_id]['size_x'],
                              model_data[class_id]['size_y'],
                              model_data[class_id]['size_z']])
            # plotting as ellipsoids
            # plot_ellipsoid(gt_pose[o], radii/2, ax, colors[o])
            # plotting as cuboids
            plot_cuboid(gt_pose[o], radii, ax, colors[o])


        # plotting estimated object pose as ellipsoids
        for k,v in aligned_quad.items():
            plot_traj([v['pose']], ax, length=200)

            plot_ellipsoid(v['pose'], v['radii'],
                           ax, colors[classes.index(labels[k])])


        ax.legend(handles=[
                Patch(facecolor=c, edgecolor=c, label=dataset_info[str(l)]) for l, c in zip(classes, colors)
            ])
        ax.grid(False)
        plt.savefig(images_save_directory + '/estimated_scene.png')
        plt.close()
        
    ################# ESTIMATED SCENE PLOTTING #################
    
    
    ################# ERROR METRICS CALCULATION #################
    
    ####### CAMERA POSE ERRORS #######
    ## 1. Root Mean Square Error(RMSE) for Trajectory Deviation (Euclidean Distance)
    # Euclidean distance
    euc_errors = []

    for i in range(len(gt_traj)):
        euc_errors.append(euclidean_distance(gt_traj[i][:3, 3], aligned_traj[i][:3, 3]))
        
    squared_error = [item ** 2 for item in euc_errors]
    
    mean_squared_error = sum(squared_error)/len(squared_error)
    
    rmse = math.sqrt(mean_squared_error)
    
    if plottings:
        fig = plt.figure(figsize=(10,6))
        ax = fig.add_subplot(111)
        ax.plot([i for i in range(len(euc_errors))], euc_errors)
        ax.set_xlabel('Camera Frame')
        ax.set_ylabel('Euclidean Distance (mm)')
        plt.title("Trajectory Deviation")
        ax.grid(False)
        plt.savefig(images_save_directory + '/trajectory_deviation.png')
        plt.close()
        
    ## 2. Average Rotation Error for Camera Poses
    rot_errors_traj = []
    
    for i in range(len(gt_traj)):
        # order is dependent
        rot_errors_traj.append(rotation_error_quaternion(matrix_to_quaternion(gt_traj[i]),
                                                         matrix_to_quaternion(aligned_traj[i])))
        
    rot_errors_traj = [value for value in rot_errors_traj if not math.isnan(value)]
    
    avg_rot_error_traj = sum(rot_errors_traj)/len(rot_errors_traj)
    
    if plottings:
        fig = plt.figure(figsize=(10,6))
        ax = fig.add_subplot(111)
        ax.plot([i for i in range(len(rot_errors_traj))], rot_errors_traj)
        ax.set_xlabel('Camera Frame')
        ax.set_ylabel('Rotation error (rad)')
        plt.title("Rotation Error")
        ax.grid(False)
        plt.savefig(images_save_directory + '/trajectory_rotation_error.png')
        plt.close()
    
    ## 3. Disparity Measure by Procrustes Analysis
    points_gt = []
    points_est = []

    for i in range(len(gt_traj)):
        points_gt.append(list(gt_traj[i][:3, 3]))
        points_est.append(list(aligned_traj[i][:3, 3]))

    points_gt = np.array(points_gt)
    points_est = np.array(points_est)

    set1, set2, disparity = procrustes_analysis(points_gt, points_est)
    
    if plottings:
        fig = plt.figure(figsize=(10,6))
        ax = fig.add_subplot(111, projection='3d')
        ax.plot(set1[:,0], set1[:,1], set1[:,2], c='r', marker='o', alpha=0.5, label='Ground Truth')
        ax.plot(set2[:,0], set2[:,1], set2[:,2], c='b', marker='o', alpha=0.5, label='Estimated')
        ax.grid(False)
        ax.legend()
        plt.title("Procrustes Analysis - Normalised and Aligned")
        plt.savefig(images_save_directory + '/procrustes_analysis.png')
        plt.close()
        
    
    ## 4. Fréchet Distance
    f_dist = frechet_distance(points_gt, points_est)
    
    ## 5. Chamfer Distance
    c_dist = chamfer_distance(points_gt, points_est)
    ####### CAMERA POSE ERRORS #######
    
    ####### OBJECT POSE ERRORS #######
    ## keep only 1 object in aligned_quad
    ## identify unique keys
    seen_integers = []
    new_labels = dict()
    unique_keys = []
    for k,v in labels.items():
        if v not in seen_integers:
            seen_integers.append(v)
            unique_keys.append(k)
            new_labels[k] = v
    labels = new_labels
    ## update the output dictionary
    new_aligned_quad = dict()
    for k,v in aligned_quad.items():
        if k in unique_keys:
            new_aligned_quad[k] = v
    aligned_quad = new_aligned_quad

    ## 1. Average Centroid Error (Euclidean Distance)
    # GT centroids
    centroids_gt = []
    for o in range(len(gt_pose)):
        centroids_gt.append(gt_pose[o][:3,3])

    # Estimated centroids
    centroids_est = []
    for k,v in aligned_quad.items():
        centroids_est.append(v['pose'][:3,3])

    centroid_errors = []

    for i in range(len(centroids_gt)):
        centroid_errors.append(euclidean_distance(centroids_gt[i], centroids_est[i]))
    # average centroid error
    avg_cen_err = sum(centroid_errors)/len(centroid_errors)
    
    
    ## 2. Average Rotation Error for Object Poses
    # rotation matrix quadrent correction
    est_obj_mod_mat = []
    
    for k,v in aligned_quad.items():
        inverse = np.eye(4)
        inverse[:3, :3] = v['pose'][:3, :3].T

        # Convert the rotation matrix to a Rotation object
        r = Rotation.from_matrix(inverse[:3, :3])

        # Convert to Euler angles (XYZ rotation order)
        euler_angles = r.as_euler('xyz', degrees=True)

        # Extract individual angles
        alpha = threshold_to_closest_multiple_of_90(euler_angles[0])  # X-axis rotation
        beta = threshold_to_closest_multiple_of_90(euler_angles[1])   # Y-axis rotation
        gamma = threshold_to_closest_multiple_of_90(euler_angles[2])  # Z-axis rotation

        rotation_transform = np.eye(4)
        # Create a Rotation matrix from Euler angles (XYZ rotation order)
        r = Rotation.from_euler('xyz', [alpha, beta, gamma], degrees=True)
        rotation_transform[:3, :3] = r.as_matrix()

        est_obj_mod_mat.append(np.dot(v['pose'], rotation_transform))
        
    # calculating the error
    rot_errors_obj = []
    

    for o in range(len(gt_pose)):
        # order is dependent
        rot_errors_obj.append(rotation_error_quaternion(matrix_to_quaternion(gt_pose[o]),
                                                        matrix_to_quaternion(est_obj_mod_mat[o])))

    
    avg_rot_error_obj = sum(rot_errors_obj)/ len(rot_errors_obj)
    
    
    ## 3. Percentage of Intersection Volume
    if plottings:
        fig = plt.figure(figsize=(10,6))
        ax = fig.add_subplot(111, projection='3d')
        ax.set_xlabel('X Axis')
        ax.set_ylabel('Y Axis')
        ax.set_zlabel('Z Axis')
        ax.grid(False)
        ax.legend(handles=[
            Patch(facecolor=c, edgecolor=c, label=dataset_info[str(l)]) for l, c in zip(classes, colors)
        ])
        
        
    # to ensure same particles are generated for oa-slam and quadricslam
    np.random.seed(50)

    ## The higher the nmber of points, the better is the monte-carlo approximation
    num_points = 10000

    overlap_cuboid = [] # percentage of overlapping points

    volume_cuboid = []
    volume_est_ellipsoid = []

    o = 0

    for k,v in aligned_quad.items():
        
        class_id = str(classes[o])
        # volume of a cuboid
        cube_dimensions = np.array([model_data[class_id]['size_x'],
                                    model_data[class_id]['size_y'],
                                    model_data[class_id]['size_z']])
        # length * breadth * height
        volume_cuboid.append(np.prod(cube_dimensions))

        # volume of an ellipsoid
        volume_est_ellipsoid.append((4/3) * np.pi * (np.prod(v['radii'])))

        # generate points within the cuboid
        points_within_cuboid = generate_points_within_cuboid(gt_pose[o][:3,3], cube_dimensions, 
                                                             gt_pose[o][:3,:3], num_points)
        points = np.array(points_within_cuboid)

        # check if the point is within the ellipsoid
        inside_est_ellipsoid = 0

        for i in range(len(points)):
            if is_point_inside_ellipsoid(points[i], v['pose'][:3,3],
                                         v['radii'], v['pose'][:3,:3]):
                inside_est_ellipsoid+=1

        overlap_cuboid.append(inside_est_ellipsoid / num_points)
        
        if plottings:
            ### Plotting the experiment ###
            # scatter the points within the plot
            ax.scatter(points[:, 0], points[:, 1], points[:, 2], c='k', s=0.1)
            # plot the GT cuboid
            plot_cuboid(gt_pose[o], cube_dimensions, ax, colors[classes.index(classes[o])])
            # plot the estimated ellipsoid
            plot_ellipsoid(v['pose'], v['radii'], ax, colors[classes.index(classes[o])])
        
        o+=1
    
    if plottings:
        plt.title("Monte-Carlo Sample Point Generation")
        plt.savefig(images_save_directory + '/intersection_volume.png')
        plt.close()
        
    intersection_percent = [] # percentage of intersection
    for i in range(len(overlap_cuboid)):
        intersection_percent.append((2*volume_cuboid[i]*overlap_cuboid[i]*100)/(volume_cuboid[i]+volume_est_ellipsoid[i]))
        
    avg_int_per = sum(intersection_percent)/len(intersection_percent)
    
    ## 4. Percentage of Intersection Volume
    if plottings:
        fig = plt.figure(figsize=(10,6))
        ax = fig.add_subplot(111, projection='3d')
        ax.set_xlabel('X Axis')
        ax.set_ylabel('Y Axis')
        ax.set_zlabel('Z Axis')
        ax.grid(False)
        ax.legend(handles=[
            Patch(facecolor=c, edgecolor=c, label=dataset_info[str(l)]) for l, c in zip(classes, colors)
        ])

    overlap_cuboid = [] # percentage of overlapping points

    volume_cuboid = []
    volume_est_ellipsoid = []

    o = 0

    for k,v in aligned_quad.items():
        
        class_id = str(classes[o])
        # volume of a cuboid
        cube_dimensions = np.array([model_data[class_id]['size_x'],
                                    model_data[class_id]['size_y'],
                                    model_data[class_id]['size_z']])
        # length * breadth * height
        volume_cuboid.append(np.prod(cube_dimensions))

        # volume of an ellipsoid
        volume_est_ellipsoid.append((4/3) * np.pi * (np.prod(v['radii'])))

        # generate points within the cuboid
        points_within_cuboid = generate_points_within_cuboid(gt_pose[o][:3,3], cube_dimensions, 
                                                             gt_pose[o][:3,:3], num_points)
        points = np.array(points_within_cuboid)

        # check if the point is within the ellipsoid
        inside_est_ellipsoid = 0
        
        
        # Estimated object aligning with GT object
        gt_inv = np.linalg.inv(gt_pose[o])
        T_rel = np.dot(gt_inv, v['pose'])
        aligned_est = np.dot(v['pose'], np.linalg.inv(T_rel)) ## NEW POSE
        # Now the length, breadth and height may interchange.
        sorted_indices = np.argsort(cube_dimensions)
        sorted_array = np.sort(v['radii'])
        aligned_size = np.ones(3)
        for j in range(3):
            aligned_size[sorted_indices[j]] = sorted_array[j] ## NEW RADIUS
            

        for i in range(len(points)):
            if is_point_inside_ellipsoid(points[i], aligned_est[:3,3],
                                         aligned_size, aligned_est[:3,:3]):
                inside_est_ellipsoid+=1

        overlap_cuboid.append(inside_est_ellipsoid / num_points)
        
        if plottings:
            ### Plotting the experiment ###
            # scatter the points within the plot
            ax.scatter(points[:, 0], points[:, 1], points[:, 2], c='k', s=0.1)
            # plot the GT cuboid
            plot_cuboid(gt_pose[o], cube_dimensions, ax, colors[classes.index(classes[o])])
            # plot the estimated ellipsoid
            plot_ellipsoid(aligned_est, aligned_size,
                           ax, colors[classes.index(classes[o])])
        
        o+=1
    
    if plottings:
        plt.title("Monte-Carlo Sample Point Generation After Alignment")
        plt.savefig(images_save_directory + '/intersection_volume_after_alignment.png')
        plt.close()
        
    intersection_percent_aligned = [] # percentage of intersection
    for i in range(len(overlap_cuboid)):
        intersection_percent_aligned.append((2*volume_cuboid[i]*overlap_cuboid[i]*100)/(volume_cuboid[i]+volume_est_ellipsoid[i]))
        
    avg_int_per_aligned = sum(intersection_percent_aligned)/len(intersection_percent_aligned)
    
    
    ####### OBJECT POSE ERRORS #######
    
    ################# ERROR METRICS CALCULATION #################
    
    ################# EXPORTING TO JSON #################
    labels = [dataset_info[str(l)] for l in classes]
    
    export_data = {'camera_pose': {
                                   'euc_error': euc_errors,
                                   'root_mean_square_error': rmse,
                                   'rotation_error': rot_errors_traj,
                                   'average_rotation_error': avg_rot_error_traj,
                                   'disparity': disparity,
                                   'frechet_distance': f_dist,
                                   'chamfer_distance': c_dist
                                    },
                   'object_pose': {
                                   'centroid_error': centroid_errors,
                                   'average_centroid_error': avg_cen_err,
                                   'rotation_error': rot_errors_obj,
                                   'average_rotation_error': avg_rot_error_obj,
                                   'volume_of_intersection': intersection_percent,
                                   'average_volume_of_intersection': avg_int_per,
                                   'volume_of_intersection_aligned': intersection_percent_aligned,
                                   'average_volume_of_intersection_aligned': avg_int_per_aligned,
                                   'labels': labels
                                    }
                   }
    if batch:
        export_path = os.path.join(dataset, 'quadric_slam_result/error_metrics_batch.json')
    else:
        export_path = os.path.join(dataset, 'quadric_slam_result/error_metrics_incre.json')
        
    with open(export_path, "w") as json_file:
        json.dump(export_data, json_file, indent=4)  # indent for pretty formatting
    ################# EXPORTING TO JSON #################

# Main Execution Block

In [14]:
##### Dataset path
dataset_path = "/home/allen/Desktop/RnD_Github/AllenIsaacRnD/dataset/"

##### Extracting the absolute path for all the datasets
datasets_path = []

for folder in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, folder)

    if os.path.isdir(folder_path) and not folder.startswith('.'):
        datasets_path.append(folder_path)
        
datasets_path.sort()
        
##### Processing each dataset
for dataset in datasets_path:
    # dataset -> path to the dataset folder
    # plottings -> the bool that controls the plots to be saved or not
    # batch -> if batch, then read the batch output file
    postprocessing(dataset, True, True)
    print("Processed: ", dataset)
    break

Processed:  /home/allen/Desktop/RnD_Github/AllenIsaacRnD/dataset/000001
